## Notebook to check VTK functionality

In [1]:
import vtk
import numpy as np
import random
from vtk.util.numpy_support import vtk_to_numpy
from math import cos, sin, pi


### Load CSD data and compute mesh

In [2]:
#Load data into VTK table object (https://github.com/Kitware/VTK/blob/master/Examples/Infovis/Python/tables3.py)
csv_source = vtk.vtkDelimitedTextReader()
csv_source.SetFieldDelimiterCharacters(",")
csv_source.SetHaveHeaders(False)
csv_source.SetDetectNumericColumns(True)
csv_source.SetFileName('/home/ntolley/Jones_Lab/lfp_reeb_local/data/gbarEvPyrAmpa_sweep/points/gbarEvPyrAmpa_sweep1.csv')
csv_source.Update()

T = csv_source.GetOutput()

# USER: vtkReebGraph's Build() function will rely on a vtkPolyData that
#  specifically only has triangles.  vtkUnstructuredGrid could be used
#  for tetrahedra.  
#  See http://www.vtk.org/doc/nightly/html/classvtkReebGraph.html#details
grid = vtk.vtkPolyData()

points = vtk.vtkPoints()
data = vtk.vtkFloatArray()
data.SetNumberOfComponents(1)
data.SetName("Function Value")

points.SetNumberOfPoints(T.GetNumberOfRows())
for r in range(T.GetNumberOfRows()):
    points.InsertPoint(r, T.GetColumn(0).GetValue(r),T.GetColumn(1).GetValue(r),T.GetColumn(2).GetValue(r))
    data.InsertNextValue(T.GetColumn(2).GetValue(r))
    

grid.SetPoints(points)
grid.GetPointData().SetScalars(data)

point_bounds = points.GetBounds()
x, y = int(point_bounds[1]), int(point_bounds[3])


tris = vtk.vtkCellArray()

for j in range(y) :
   for i in range(x) :
      tri = [j*x+i, j*x+i+1, (j+1)*x+i]
      tris.InsertNextCell(3, tri)
      
      tri = [j*x+i+1, (j+1)*x+i, (j+1)*x+i+1]
      tris.InsertNextCell(3, tri)

grid.SetPolys(tris)

### Simplify mesh with decimate

In [3]:
decimate = vtk.vtkDecimatePro()
decimate.SetInputData(grid)
decimate.SetTargetReduction(0.99)
decimate.PreserveTopologyOn()
decimate.Update()

decimated = vtk.vtkPolyData()
decimated.ShallowCopy(decimate.GetOutput())


data_decimated = vtk.vtkFloatArray()
data_decimated.SetNumberOfComponents(1)
data_decimated.SetName("Function Value")


### Build Reeb Graph with decimated mesh

In [4]:
reeb_graph = vtk.vtkReebGraph()
err = reeb_graph.Build(decimated, decimated.GetPointData().GetScalars())

### Iterates over all edges in graph and stores in list

In [5]:
iter_edge = vtk.vtkEdgeListIterator()
iter_edge.SetGraph(reeb_graph)

edge_list = []
while iter_edge.HasNext():
    temp_edge = iter_edge.NextGraphEdge()
    source = temp_edge.GetSource()
    target = temp_edge.GetTarget()
    edge_list.append([source, target])

### Iterates over nodes and stores x,y,z position

In [58]:
iter_vertex = vtk.vtkVertexListIterator()
iter_vertex.SetGraph(reeb_graph)

# Vertex coordinates are stored under original mesh ID's 
point_data = decimated.GetPoints().GetData()
vertex_mapping = reeb_graph.GetVertexData().GetAbstractArray(0)

vertex_list = []
while iter_vertex.HasNext():
    graph_vertex_id = iter_vertex.Next()
    mesh_vertex_id = int(vertex_mapping.GetTuple(graph_vertex_id)[0])

    vertex_pos = point_data.GetTuple(mesh_vertex_id) 
    vertex_list.append(list(vertex_pos))


In [59]:
print()